In [1]:
!pip install -r lab3/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.6/200.6 kB 3.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.2/391.2 kB 9.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.

In [149]:
import requests
from PIL import Image

file_path = "datasets/coco128/images/val2017/000000119995.jpg"

with open(file_path, 'rb') as file:
    data = file.read()
    
def run_bento(data):
    headers = {
        "accept": "application/json",
        "Content-Type": "image/png",
    }
    
    response = requests.post(
        "http://0.0.0.0:3000/invocation",
        headers=headers,
        data=data,
    )
    
    return response.text

In [150]:
def calculate_mAP(predictions, annotations, image_width, image_height):
    
    # Convert YOLO format to (x_min, y_min, x_max, y_max)
    def yolo_to_bbox(x_center, y_center, width, height, img_width, img_height):
        x_min = (x_center - width / 2) * img_width
        y_min = (y_center - height / 2) * img_height
        x_max = (x_center + width / 2) * img_width
        y_max = (y_center + height / 2) * img_height
        return [x_min, y_min, x_max, y_max]
    
    
    # Convert predictions to (x_center, y_center, width, height)
    def pred_to_bbox(pred):
        x_min, y_min, width, height = pred
        x_max = x_min + width
        y_max = y_min + height
        return [x_min, y_min, x_max, y_max]
    
    
    # Calculate Intersection over Union (IoU)
    def calculate_iou(box1, box2):
        x1_min, y1_min, x1_max, y1_max = box1
        x2_min, y2_min, x2_max, y2_max = box2
    
        inter_x_min = max(x1_min, x2_min)
        inter_y_min = max(y1_min, y2_min)
        inter_x_max = min(x1_max, x2_max)
        inter_y_max = min(y1_max, y2_max)
    
        inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)
        box1_area = (x1_max - x1_min) * (y1_max - y1_min)
        box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    
        union_area = box1_area + box2_area - inter_area
        iou = inter_area / union_area
        return iou
    
    
    # Process ground truths
    gt_bboxes = []
    for ann in annotations:
        class_id, x_center, y_center, width, height = ann
        bbox = yolo_to_bbox(x_center, y_center, width, height, image_width, image_height)
        gt_bboxes.append((class_id, bbox))
    
    # Process predictions
    pred_bboxes = []
    for pred, class_id in predictions:
        bbox = pred_to_bbox(pred)
        pred_bboxes.append((class_id, bbox))
    
    # Calculate IoUs and determine TP, FP, FN
    iou_threshold = 0.5
    tp, fp, fn = 0, 0, 0
    
    # Create a list to store matched ground truths
    matched_gts = set()
    
    for pred_class, pred_bbox in pred_bboxes:
        if pred_class != 15:
            continue
        matched = False
        for gt_idx, (gt_class, gt_bbox) in enumerate(gt_bboxes):
            iou = calculate_iou(pred_bbox, gt_bbox)
            print(iou, pred_class, gt_class)
            if iou >= iou_threshold and pred_class == gt_class and gt_idx not in matched_gts:
                tp += 1
                matched_gts.add(gt_idx)
                matched = True
                break
        if not matched:
            fp += 1
    
    # Calculate FN
    fn = len(gt_bboxes) - tp
    
    # Calculate Precision and Recall
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    mAP = precision

    # Assuming we have one precision-recall point, mAP is simply precision at this recall
    return precision


In [151]:
import json
def convert_predictions(pred_dicts_str):
    pred_dicts = json.loads(pred_dicts_str)
    
    # Проходим по каждому словарю в списке и извлекаем данные
    predictions = []
    for item in pred_dicts:
        bbox = item["bbox"]
        class_id = item["class"]
        predictions.append((bbox, class_id))
    return predictions

def convert_annotations(annotations):
    # Разбиваем строку на строки, используя символ новой строки в качестве разделителя
    lines = annotations.strip().split('\n')
    
    # Создаём список для хранения кортежей
    data_list = []
    
    # Проходим по каждой строке
    for line in lines:
        # Разбиваем строку на подстроки, используя пробел в качестве разделителя
        values = line.split()
        # Преобразуем значения в числа
        values = [float(value) for value in values]
        # Преобразуем в кортеж и добавляем в список
        data_list.append(tuple(values))
    
    return data_list

In [154]:
import os

def process_files_and_read_labels():
    images_directory = "datasets/cats_dataset/cats_dataset/images/val"
    labels_directory = "datasets/cats_dataset/cats_dataset/labels/val"

    # Получаем список файлов в директории с изображениями
    image_files = os.listdir(images_directory)
    i = 0
    mAP_sum = 0
    # Проходим по каждому файлу изображения
    for image_file in image_files:
        # Проверяем, что файл является файлом изображения
        if image_file.endswith((".jpg")):

            with open(os.path.join(images_directory, image_file), 'rb') as file:
                data = file.read()
            
            image = Image.open(os.path.join(images_directory, image_file))
            width, height = image.size
            
            predictions = convert_predictions(run_bento(data))
         
            label_file = os.path.join(labels_directory, os.path.splitext(image_file)[0] + ".txt")

            
            i = i+1
            
            with open(label_file, "r") as f:
                label_content = f.read()
            mAP = calculate_mAP(predictions, convert_annotations(label_content), width, height)
            mAP_sum = mAP_sum + mAP
                
          
            
    return mAP_sum/45

# Вызываем функцию для обработки файлов и чтения соответствующих меток
round(process_files_and_read_labels(),2)

0.6457153322303872 15 15.0
0.9599627452450264 15 15.0
0.9380434597567834 15 15.0
0.7493950786803572 15 15.0
0.9209483022681416 15 15.0
0.9528520416993599 15 15.0
0.852423164934648 15 15.0
0.955915425380864 15 15.0
0.8955318409013261 15 15.0
0.364403579218337 15 15.0
0.5609177230878107 15 15.0
0.7881525948319493 15 15.0
0.6977114717016342 15 15.0
0.5558050646290761 15 15.0
0.9336268554417606 15 15.0
0.7051771818876917 15 15.0
0.9456073260073261 15 15.0
0.9192767830797457 15 15.0
0.015491640975389614 15 15.0
0.9347282678075123 15 15.0
0.8335908317235056 15 15.0
0.884712559455704 15 15.0
0.8933503221577023 15 15.0
0.8820452796020122 15 15.0
0.949918574659736 15 15.0
0.9635915720066117 15 15.0
0.7985214160555546 15 15.0
0.9324623650007765 15 15.0
0.7996844798270977 15 15.0
0.31273624879414885 15 15.0
0.8604326268118332 15 15.0
0.8836298156248383 15 15.0
0.9398986374672345 15 15.0
0.7838446801334686 15 15.0
0.9776799904640815 15 15.0
0.9227587545787543 15 15.0
0.9307410129649507 15 15.0
0.3

0.71